In [28]:
import ee
import os
import pandas as pd
from datetime import datetime, timedelta
from src.controllers.time_series_hls import HLS
from concurrent.futures import ThreadPoolExecutor
from src.controllers.sentinel_ import Sentinel2Processor
from src.controllers.geometry import GeoDataFrameProcessor
from src.controllers.eras import PrecipitationTemperatureRadiationData

# Conecta ao Google Earth Engine
ee.Authenticate()
ee.Initialize()

# Datas de análise
start_date = '2023-01-01'
end_date = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

#Paht polygons
path = r"C:\Users\ander\OneDrive\Área de Trabalho\msu\bases\base\fields_tests.geojson"

vegetation_index = 'ndvi'

#Order NDVI
order_ndvi = 30

window_size = 7

poly_order = 3

product = 's2'

#Read geometry file
processador = GeoDataFrameProcessor(path)

In [ ]:
def update_start_date(file_path, default_start_date, id):

    file_ndvi = os.path.join(file_path, "ndvi_df.csv")
    file_weather = os.path.join(file_path, "ndvi_df.csv")

    if not file_ndvi:
        df = pd.read_csv(file_ndvi)
        default_start_date_ndvi = pd.to_datetime(df.sort_values(by='date')['date'].values[-1]).strftime('%Y-%m-%d')
    else:
        default_start_date_ndvi = default_start_date

    if not file_weather:
        df = pd.read_csv(file_weather)
        default_start_date_weather = pd.to_datetime(df.sort_values(by='date')['date'].values[-1]).strftime('%Y-%m-%d')
    else:
        default_start_date_weather = default_start_date

    return default_start_date_weather, default_start_date_ndvi



,date,id,ndvi,satellite,timestamps
0,2023-01-05,T14TPP_20230105T171150,-0.054202,landsat,2023-01-05
1,2023-01-06,T14TPP_20230106T170536,-0.054073,landsat,2023-01-06
2,2023-01-07,NaN,-0.055007,NaN,2023-01-07
3,2023-01-08,NaN,-0.055942,NaN,2023-01-08
4,2023-01-09,NaN,-0.056876,NaN,2023-01-09
...,...,...,...,...,...
668,2024-11-03,NaN,0.271187,NaN,2024-11-03
669,2024-11-04,NaN,0.263209,NaN,2024-11-04
670,2024-11-05,NaN,0.255231,NaN,2024-11-05
671,2024-11-06,NaN,0.247253,NaN,2024-11-06


In [43]:
# Função que processa cada polígono individualmente
vertices, geometry = processador.extract_coordinates(10)

if product == 'hls':
    satellite = HLS(geometry, start_date, end_date)
    ndvi_df = satellite.convert_to_dataframe()
else:
    s2 = Sentinel2Processor( start_date, end_date, geometry,  cloud_threshold=0)
    ndvi_df = s2.get_filtered_df()

'2024-11-12'

In [ ]:


def process_poligono(id_poligono, product, start_date, end_date ):
    # Função que processa cada polígono individualmente
    vertices, geometry = processador.extrair_coordenadas(id_poligono)

    if product == 'hls':
        satellite = HLS(geometry, start_date, end_date)
        ndvi_df = satellite.convert_to_dataframe()
    else:
        s2 = Sentinel2Processor( start_date, end_date, geometry,  cloud_threshold=35)
        ndvi_df = s2.get_filtered_df()


    # Obtenção de latitude e longitude (exemplo)
    lat = geometry.getInfo()['coordinates'][0][0][1]
    lon = geometry.getInfo()['coordinates'][0][0][0]

    # Obtenção dos dados de precipitação, temperatura e radiação
    data = PrecipitationTemperatureRadiationData(lat, lon, start_date, end_date)
    df = data.get_dataframe()

    # Criar diretório e salvar DataFrames
    directory = f"data_/{id_poligono}"
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Salvar DataFrames
    ndvi_df.to_csv(os.path.join(directory, "ndvi_df.csv"), index=False)
    df.to_csv(os.path.join(directory, "precipitation_temperature_data.csv"), index=False)

    print(f"Processamento concluído para id_poligono {id_poligono}")

for i in processador.index_polygons:
    process_poligono(i, product, start_date, end_date)



In [2]:
import ee
import os
import pandas as pd
from datetime import datetime, timedelta
from src.controllers.time_series_hls import HLS
from src.controllers.sentinel_ import Sentinel2Processor
from src.controllers.geometry import GeoDataFrameProcessor
from src.controllers.eras import PrecipitationTemperatureRadiationData

# Connect to Google Earth Engine
ee.Authenticate()
ee.Initialize()

# Analysis Dates
end_date = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

# Paths and Settings
path_polygons = r"C:\Users\ander\OneDrive\Área de Trabalho\msu\bases\base\fields_tests.geojson"
ndvi_order = 30
window_size = 7
poly_order = 3
product = 's2'

# Read geometry file
geo_processor = GeoDataFrameProcessor(path_polygons)

def update_start_date(file_path, default_start_date):
    """Update start date to the last available date in existing file."""
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        if 'date' in df.columns:
            last_date = pd.to_datetime(df['date']).max()
            return (last_date + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
    return default_start_date

def process_polygon(polygon_id, product, start_date, end_date):
    """Process each polygon individually."""
    vertices, geometry = geo_processor.extract_coordinates(polygon_id)

        # Save DataFrames in structured directory
    output_directory = f".data_/{polygon_id}"

        # File paths for ndvi and weather data
    ndvi_file = os.path.join(output_directory, "ndvi_df.csv")
    weather_file = os.path.join(output_directory, "precipitation_temperature_data.csv")

        # Adjust start date based on existing files
    start_date_ndvi = update_start_date(ndvi_file, start_date)
    start_date_wather = update_start_date(weather_file, start_date)

    print(start_date_ndvi, start_date_wather, end_date)

    # Satellite data processing
    if product == 'hls':
        satellite = HLS(geometry, start_date_ndvi, end_date)
        ndvi_df = satellite.convert_to_dataframe()
    else:
        sentinel = Sentinel2Processor(start_date_ndvi, end_date, geometry, cloud_threshold=35)
        ndvi_df = sentinel.get_filtered_df()

    print(ndvi_df)

    # Get latitude and longitude
    lat, lon = geometry.getInfo()['coordinates'][0][0][1], geometry.getInfo()['coordinates'][0][0][0]

    # Get precipitation, temperature, and radiation data
    weather_data = PrecipitationTemperatureRadiationData(lat, lon, start_date_wather, end_date)
    weather_df = weather_data.get_dataframe()

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Save DataFrames
    ndvi_df.to_csv(ndvi_file, index=False)
    weather_df.to_csv(weather_file, index=False)

    print(f"Processing completed for polygon_id {polygon_id}")

# Process all polygons
for polygon_id in geo_processor.index_polygons:
    process_polygon(polygon_id, product, '2023-01-01', end_date)


2024-11-13 2024-11-07 2024-11-14
Empty DataFrame
Columns: [date, id, ndvi, satellite, timestamps]
Index: []


KeyError: 'date'